[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/swiftsoftwaregroup/tropess-notes-python) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/swiftsoftwaregroup/tropess-notes-python/blob/main/book/quick-start-gridded-plot-methane-column.ipynb) [![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/swiftsoftwaregroup/tropess-notes-python/blob/main/book/quick-start-gridded-plot-methane-column.ipynb)

# Methane (CH4) Partial Column - Gridded Plot (1&deg; x 1&deg;)

Create a gridded plot of Methane CH4 partial column retrieved from the [Cross-track Infrared Sounder (CrIS JPSS-1)](https://www.nesdis.noaa.gov/current-satellite-missions/currently-flying/joint-polar-satellite-system/cross-track-infrared) global observations.

In [ ]:
import numpy as np
from netCDF4 import Dataset

import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap

from scipy.interpolate import griddata
from scipy.spatial import cKDTree

In [ ]:

# Open the netCDF file
dataset = Dataset('./data/TROPESS_CrIS-JPSS1_L2_Summary_CH4_20230516_MUSES_R1p20_FS_F0p6.nc', 'r')

# Read the data from your variables
latitude = dataset.variables['latitude'][:]
longitude = dataset.variables['longitude'][:]
x_col_p = dataset.variables['x_col_p'][:]

dataset.close()

# Specify figure size (in inches)
dpi=300; w=3600 / dpi; h=2400 / dpi
plt.figure(figsize=(w, h))  

# Create a basemap instance
m = Basemap(projection='cyl', resolution='l',
            llcrnrlat=-90, urcrnrlat=90,  # set latitude limits to -90 and 90
            llcrnrlon=-180, urcrnrlon=180)  # set longitude limits to -180 and 180

m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=0.5)

# Draw parallels (latitude lines) and meridians (longitude lines)
parallels = np.arange(-90., 91., 30.)
m.drawparallels(parallels, labels=[True,False,False,False], linewidth=0.3)

meridians = np.arange(-180., 181., 60.)
m.drawmeridians(meridians, labels=[False,False,False,True], linewidth=0.3)

# Get the grid for the interpolated values
grid_lat, grid_lon = np.mgrid[-90:91:1, -180:181:1]

# Interpolate the data using griddata
grid_x_col_p = griddata((latitude, longitude), x_col_p, (grid_lat, grid_lon), method='linear')

# Find the distance to the nearest original point for each point in the interpolated grid
tree = cKDTree(np.vstack((latitude, longitude)).T)
dist, _ = tree.query(np.vstack((grid_lat.ravel(), grid_lon.ravel())).T)

# Reshape the distance array to have the same shape as the x_col_p grid
dist_grid = dist.reshape(grid_x_col_p.shape)

# Mask the interpolated values that are too far from any original point
max_distance_degrees = 3.0
grid_x_col_p[dist_grid > max_distance_degrees] = np.nan

# Plot the interpolated data using pcolormesh instead of scatter
sc = m.pcolormesh(grid_lon, grid_lat, grid_x_col_p, latlon=True, cmap='jet')

# Add a colorbar
cbar = m.colorbar(sc, location='bottom', pad="10%")
cbar.set_label('x_col_p')

plt.show()

Bias corrected column-averaged dry air mixing ratio of Methane for the partial column (x_col_p) from 826 hPa to Top of Atmosphere (TOA) in ppbv - May 16th, 2023. 